In [1]:
!pip install --upgrade -qq git+https://github.com/huggingface/diffusers.git transformers accelerate

In [2]:
# Text2Img Generation

import torch 
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

device = "cuda"

# 파이프라인 불러오기
repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

In [3]:
prompt = str(input('prompt: '))  # 프롬프트를 영어로 입력해보세요

In [4]:
# 이미지 저장 폴더 만들기   
import os
# os.mkdir("/aiffel/aiffel/diffusers")    # diffusers 폴더를 이미 만들었다면 주석 처리해 주세요.

# 입력한 프롬프트를 사용하여 이미지 생성 
image = pipe(prompt, num_inference_steps=20, guidance_scale=7).images[0]

# 이미지 저장
image.save("/aiffel/aiffel/diffusers/image9.png")  

# 이미지 출력 
image

In [5]:
# GPU 메모리를 지우는 코드입니다.
import torch
torch.cuda.empty_cache()

# 인체 자세 감지

In [8]:
# controlnet-aux를 설치합니다. Human pose를 검출해주는 controlnet의 보조용 모델입니다.
!pip install controlnet-aux==0.0.1

     |████████████████████████████████| 44 kB 2.9 MB/s             


In [9]:
from diffusers.utils import load_image

openpose_image = load_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/person.png"
)
openpose_image

In [10]:
# Openpose 검출기를 사용하여 자세만 추출

from controlnet_aux import OpenposeDetector

# 인체의 자세를 검출하는 사전 학습된 ControlNet 불러오기
openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

# 이미지에서 자세 검출
openpose_image = openpose(openpose_image)
openpose_image


In [11]:
# 생성된 포즈 이미지를 저장
openpose_image.save('/aiffel/aiffel/diffusers/human_pose.png')
